In [1]:
from src import Ofstat, MarketValue, Understat as my_understat
from src.Season import Season, get_season
from src.Understat import table_team_filter, enum_to_dict, reformat
import pandas as pd
from selenium import webdriver
from understat import Understat
import aiohttp
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
temp_table = pd.read_csv('final_py_test.csv')

In [3]:
base_url ='https://ofstats.com/'
matches_url = 'https://ofstats.com/matches/view/'
seasons_url = 'https://ofstats.com/season/'

In [4]:
last_dataset_path = '20_21_w_fixed.csv' #20_21_w_fixed.csv
last_dataset = pd.read_csv(last_dataset_path)

In [5]:
# какую лигу и сезон хотим спарсить
liga = 'epl'
season = get_season("2022-2023")

In [5]:
# проверить, на какую дату в датасете последние данные
# это все надо спрятать
table_season = Ofstat.get_season(liga, season)
last_dataset_liga_season = last_dataset[last_dataset['Season'] == table_season]
try:
    last_match_date = sorted([datetime.strptime(x,'%d.%m.%Y') for x in last_dataset_liga_season['Date']])[-1] 
    print(f'Latest match in {table_season} was on {last_match_date}')
except IndexError:
    print(f'No data on season {table_season} in {last_dataset_path}')


No data on season england-premier-league-2022-2023 in 20_21_w_fixed.csv


In [6]:
# получаем таблицу матчей лига-сезон
session = aiohttp.ClientSession()
understat = Understat(session)
fixtures = await understat.get_league_fixtures("epl", 2022)
session.close()

<coroutine object ClientSession.close at 0x0000017A3A09A6C0>

In [7]:
# фильтруем матчи по дате: последней дате из датасета, сегодняшней дате, любой дате

starting_date = datetime.today() # or last_match_date

fixtures = pd.DataFrame(fixtures)
fixtures = fixtures[pd.to_datetime(fixtures['datetime'], format='%Y.%m.%d') > starting_date]
fixtures.head()

,id,isResult,h,a,goals,xG,datetime
14,18362,False,"{'id': '83', 'title': 'Arsenal', 'short_title'...","{'id': '81', 'title': 'West Ham', 'short_title...","{'h': None, 'a': None}","{'h': None, 'a': None}",2022-12-26 12:00:00
15,18363,False,"{'id': '71', 'title': 'Aston Villa', 'short_ti...","{'id': '87', 'title': 'Liverpool', 'short_titl...","{'h': None, 'a': None}","{'h': None, 'a': None}",2022-12-26 12:00:00
16,18364,False,"{'id': '244', 'title': 'Brentford', 'short_tit...","{'id': '82', 'title': 'Tottenham', 'short_titl...","{'h': None, 'a': None}","{'h': None, 'a': None}",2022-12-26 12:00:00
17,18365,False,"{'id': '80', 'title': 'Chelsea', 'short_title'...","{'id': '73', 'title': 'Bournemouth', 'short_ti...","{'h': None, 'a': None}","{'h': None, 'a': None}",2022-12-26 12:00:00
18,18366,False,"{'id': '78', 'title': 'Crystal Palace', 'short...","{'id': '228', 'title': 'Fulham', 'short_title'...","{'h': None, 'a': None}","{'h': None, 'a': None}",2022-12-26 12:00:00


In [8]:
# форматируем данные из апи
# колонки, алиасы

_host = [x['title'] for x in fixtures['h']]
_guest = [x['title'] for x in fixtures['a']]
_date = [x.strftime("%d.%m.%Y") for x in pd.to_datetime(fixtures['datetime'], format='%Y.%m.%d')]
_status = 'Upcoming'
_season = Ofstat.get_season(liga, season)

temp_table = pd.DataFrame({'Date' : _date , 'Host': _host, 'Guest': _guest, 'Status': _status, 'Season' : _season})
temp_table.head()

,Date,Host,Guest,Status,Season
0,26.12.2022,Arsenal,West Ham,Upcoming,england-premier-league-2022-2023
1,26.12.2022,Aston Villa,Liverpool,Upcoming,england-premier-league-2022-2023
2,26.12.2022,Brentford,Tottenham,Upcoming,england-premier-league-2022-2023
3,26.12.2022,Chelsea,Bournemouth,Upcoming,england-premier-league-2022-2023
4,26.12.2022,Crystal Palace,Fulham,Upcoming,england-premier-league-2022-2023


In [9]:
# инитим драйвер
# когда откроется окно firefox, на него нужно нажать
driver = webdriver.Firefox()

In [13]:
# получаем статистику offstat за t-1 сезон: 
# Host: Average shots stvor, Guest: Average shots stvor, Host: Average corner situation, Guest: Average corner situation
# Host: SCR, Guest: SCR, Host: Ball control, Guest: Ball control

In [10]:
# чтобы парсить Ofstat после Understat, нужно применить алиасы
temp_table['Host'] = temp_table['Host'].replace(enum_to_dict(Ofstat.AliasesUnderstatToOfstat)[liga])
temp_table['Guest'] = temp_table['Guest'].replace(enum_to_dict(Ofstat.AliasesUnderstatToOfstat)[liga])

In [12]:
temp_table = Ofstat.get_matches_statistics(driver, temp_table, matches_url)
temp_table.head()

NoSuchElementException: Message: Unable to locate element: //*[@href="/matches/statistics/arsenal-west-ham-united-2022-12-26"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:394:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:280:16


In [16]:
# получаем market value
# Host: market value
# Guest: maerket value

In [11]:
# season_num = str(season.value)
temp_table = MarketValue.get_market_value_with_alias(temp_table, driver, liga, season, MarketValue.AliasesOfstatToMarket)
temp_table.head()

Error: Wolverhampton Wanderers Wanderers


,Date,Host,Guest,Status,Season,Host: current season market value,Guest: current season market value
0,26.12.2022,Arsenal,West Ham United,Upcoming,england-premier-league-2022-2023,€749.00m,€459.50m
1,26.12.2022,Aston Villa,Liverpool,Upcoming,england-premier-league-2022-2023,€464.60m,€867.00m
2,26.12.2022,Brentford,Tottenham Hotspur,Upcoming,england-premier-league-2022-2023,€305.50m,€678.40m
3,26.12.2022,Chelsea,Bournemouth,Upcoming,england-premier-league-2022-2023,€846.50m,€179.70m
4,26.12.2022,Crystal Palace,Fulham,Upcoming,england-premier-league-2022-2023,€293.20m,€240.00m


In [12]:
# чтобы парсить Understat после Offstat, нужно применить алиасы
# чтобы парсить Ofstat после Understat, нужно применить алиасы
temp_table['Host'] = temp_table['Host'].replace(enum_to_dict(my_understat.AliasesOfstatToUnderstat)[liga])
temp_table['Guest'] = temp_table['Guest'].replace(enum_to_dict(my_understat.AliasesOfstatToUnderstat)[liga])

In [12]:
uliga = my_understat.get_liga(liga) #'EPL' #'Ligue_1' #Serie_A #EPL

#берем на 1 сезон раньше текущего

t_minus_1_season= int(Ofstat.season_mapping[season.value-1].split('-')[0])
session = aiohttp.ClientSession()
understat = Understat(session)

league_table =  await understat.get_league_table(league_name=uliga, season=t_minus_1_season, with_headers=True)

session.close()

league_table = my_understat.trim_async_table(league_table)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000195689B8F70>


In [14]:
# получаем статистику за домашние игры из Understat api
session = aiohttp.ClientSession()
understat = Understat(session)

league_table_h =  await understat.get_league_table(league_name=uliga, season=t_minus_1_season, with_headers=True, h_a='home')

session.close()

league_table_h = my_understat.trim_async_table(league_table_h)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000198EADE5F10>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x00000198EAE127C0>, 102249.046)]']
connector: <aiohttp.connector.TCPConnector object at 0x00000198EADE5EE0>


In [15]:
# получаем статистику за выездные игры из Understat api
session = aiohttp.ClientSession()
understat = Understat(session)

league_table_a =  await understat.get_league_table(league_name=uliga, season=t_minus_1_season, with_headers=True,  h_a='away')

session.close()

league_table_a = my_understat.trim_async_table(league_table_a)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000198EAF08E80>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x00000198EABDF820>, 102250.328)]']
connector: <aiohttp.connector.TCPConnector object at 0x00000198EAF08EB0>


In [18]:
# Хозяева: считаем показатели и линкуем их к temp_table
# 'Host: G average at home last season', 'Host: G average as guests last season',  'Host: GA average last season'
# 'Host: G average last season',  'Host: league positon last season',  'Host: GA 1-15 min last season'
# 'Host: GA 16-30 min last season', 'Host: GA 31-45 min last season', 'Host: GA 46-60 min last season'
# 'Host: GA 61-75 min last season', 'Host: GA 76+ min last season'

In [19]:
for index, row in temp_table.iterrows():
    print(index, row['Host'], row['Guest'], row['Date'])
    
    try:
        host_stats = await understat.get_team_stats(row['Host'], t_minus_1_season)

        host_stats=pd.DataFrame(host_stats)

        temp_table.loc[index, 'Host: G average at home last season'] = (table_team_filter(league_table_h, row['Host'])['G'] / table_team_filter(league_table_h, row['Host'])['M']).values[0]
        temp_table.loc[index, 'Host: G average as guests last season'] =  (table_team_filter(league_table_a, row['Host'])['G'] / table_team_filter(league_table_a, row['Host'])['M']).values[0]

        temp_table.loc[index, 'Host: GA average last season'] = (table_team_filter(league_table, row['Host'])['GA'] / table_team_filter(league_table, row['Host'])['M']).values[0]


        temp_table.loc[index, 'Host: G average last season'] = (table_team_filter(league_table, row['Host'])['G'] / table_team_filter(league_table, row['Host'])['M']).values[0]

        temp_table.loc[index, 'Host: league positon last season'] = table_team_filter(league_table, row['Host']).index.values[0]

        temp_table.loc[index, 'Host: GA 1-15 min last season'] =  (host_stats['timing']['1-15']['goals']/table_team_filter(league_table, row['Host'])['G']).values[0]
        temp_table.loc[index, 'Host: GA 16-30 min last season'] =  (host_stats['timing']['16-30']['goals']/table_team_filter(league_table, row['Host'])['G']).values[0]
        temp_table.loc[index, 'Host: GA 31-45 min last season'] =  (host_stats['timing']['31-45']['goals']/table_team_filter(league_table, row['Host'])['G']).values[0]
        temp_table.loc[index, 'Host: GA 46-60 min last season'] =  (host_stats['timing']['46-60']['goals']/table_team_filter(league_table, row['Host'])['G']).values[0]
        temp_table.loc[index, 'Host: GA 61-75 min last season'] =  (host_stats['timing']['61-75']['goals']/table_team_filter(league_table, row['Host'])['G']).values[0]
        temp_table.loc[index, 'Host: GA 76+ min last season'] =  (host_stats['timing']['76+']['goals']/table_team_filter(league_table, row['Host'])['G']).values[0]
        session.close()
    except:
        print('Error',row['Host'])
        session.close()

0 Arsenal West Ham 26.12.2022
1 Aston Villa Liverpool 26.12.2022
2 Brentford Tottenham 26.12.2022
3 Chelsea Bournemouth 26.12.2022
4 Crystal Palace Fulham 26.12.2022
5 Everton Wolverhampton Wanderers 26.12.2022
6 Leeds Manchester City 26.12.2022
7 Leicester Newcastle United 26.12.2022
8 Manchester United Nottingham Forest 26.12.2022
9 Southampton Brighton 26.12.2022
10 Bournemouth Crystal Palace 31.12.2022
Error Bournemouth
11 Brighton Arsenal 31.12.2022
12 Fulham Southampton 31.12.2022
Error Fulham
13 Liverpool Leicester 31.12.2022
14 Manchester City Everton 31.12.2022
15 Newcastle United Leeds 31.12.2022
16 Nottingham Forest Chelsea 31.12.2022
Error Nottingham Forest
17 Tottenham Aston Villa 31.12.2022
18 West Ham Brentford 31.12.2022
19 Wolverhampton Wanderers Manchester United 31.12.2022
20 Arsenal Newcastle United 02.01.2023
21 Aston Villa Wolverhampton Wanderers 02.01.2023
22 Brentford Liverpool 02.01.2023
23 Chelsea Manchester City 02.01.2023
24 Crystal Palace Tottenham 02.01.20

Error Bournemouth
201 Brighton Southampton 20.05.2023
202 Fulham Crystal Palace 20.05.2023
Error Fulham
203 Liverpool Aston Villa 20.05.2023
204 Manchester City Chelsea 20.05.2023
205 Newcastle United Leicester 20.05.2023
206 Nottingham Forest Arsenal 20.05.2023
Error Nottingham Forest
207 Tottenham Brentford 20.05.2023
208 West Ham Leeds 20.05.2023
209 Wolverhampton Wanderers Everton 20.05.2023
210 Arsenal Wolverhampton Wanderers 28.05.2023
211 Aston Villa Brighton 28.05.2023
212 Brentford Manchester City 28.05.2023
213 Chelsea Newcastle United 28.05.2023
214 Crystal Palace Nottingham Forest 28.05.2023
215 Everton Bournemouth 28.05.2023
216 Leeds Tottenham 28.05.2023
217 Leicester West Ham 28.05.2023
218 Manchester United Fulham 28.05.2023
219 Southampton Liverpool 28.05.2023


In [21]:
# Гости: считаем показатели и линкуем их к temp_table
# 'Guest: G average at home last season', 'Guest: G average as guests last season', 'Guest: G average last season'
# 'Guest: GA average last season', 'Guest: league position last season', 'Guest: GA 1-15 min last season'
# 'Guest: GA 16-30 min last season',  'Guest: GA 31-45 min last season', 'Guest: GA 46-60 min last season'
# 'Guest: GA 61-75 min last season', 'Guest: GA 76+ min last season'

In [30]:
for index, row in temp_table.iterrows():
    print(index, row['Host'], row['Guest'], row['Date'])
    try:
        guest_stats = await understat.get_team_stats(row['Guest'], t_minus_1_season)

        temp_table.loc[index, 'Guest: G average at home last season'] = (table_team_filter(league_table_h, row['Guest'])['G'] / table_team_filter(league_table_h, row['Guest'])['M']).values[0]
        temp_table.loc[index, 'Guest: G average as guests last season'] =  (table_team_filter(league_table_a, row['Guest'])['G'] / table_team_filter(league_table_a, row['Guest'])['M']).values[0]

        temp_table.loc[index, 'Guest: G average last season'] = (table_team_filter(league_table, row['Guest'])['G'] / table_team_filter(league_table, row['Guest'])['M']).values[0]

        temp_table.loc[index, 'Guest: GA average last season'] = (table_team_filter(league_table, row['Guest'])['GA'] / table_team_filter(league_table, row['Guest'])['M']).values[0]
        temp_table.loc[index, 'Guest: league position last season'] = table_team_filter(league_table, row['Guest']).index.values[0]


        temp_table.loc[index, 'Guest: GA 1-15 min last season'] =  (guest_stats['timing']['1-15']['goals']/table_team_filter(league_table, row['Guest'])['G']).values[0]
        temp_table.loc[index, 'Guest: GA 16-30 min last season'] = (guest_stats['timing']['16-30']['goals']/table_team_filter(league_table, row['Guest'])['G']).values[0]
        temp_table.loc[index, 'Guest: GA 31-45 min last season'] = (guest_stats['timing']['31-45']['goals']/table_team_filter(league_table, row['Guest'])['G']).values[0]
        temp_table.loc[index, 'Guest: GA 46-60 min last season'] = (guest_stats['timing']['46-60']['goals']/table_team_filter(league_table, row['Guest'])['G']).values[0]
        temp_table.loc[index, 'Guest: GA 61-75 min last season'] = (guest_stats['timing']['61-75']['goals']/table_team_filter(league_table, row['Guest'])['G']).values[0]
        temp_table.loc[index, 'Guest: GA 76+ min last season'] = (guest_stats['timing']['76+']['goals']/table_team_filter(league_table, row['Guest'])['G']).values[0]
        print(row['Guest'], 'OK')
        session.close()
    except:
        print('Error', row['Guest'])
        session.close()

0 Arsenal West Ham 26.12.2022
West Ham OK
1 Aston Villa Liverpool 26.12.2022
Liverpool OK
2 Brentford Tottenham 26.12.2022
Tottenham OK
3 Chelsea Bournemouth 26.12.2022
Error Bournemouth
4 Crystal Palace Fulham 26.12.2022
Error Fulham
5 Everton Wolverhampton Wanderers 26.12.2022
Wolverhampton Wanderers OK
6 Leeds Manchester City 26.12.2022
Manchester City OK
7 Leicester Newcastle United 26.12.2022
Newcastle United OK
8 Manchester United Nottingham Forest 26.12.2022
Error Nottingham Forest
9 Southampton Brighton 26.12.2022
Brighton OK
10 Bournemouth Crystal Palace 31.12.2022
Crystal Palace OK
11 Brighton Arsenal 31.12.2022
Arsenal OK
12 Fulham Southampton 31.12.2022
Southampton OK
13 Liverpool Leicester 31.12.2022
Leicester OK
14 Manchester City Everton 31.12.2022
Everton OK
15 Newcastle United Leeds 31.12.2022
Leeds OK
16 Nottingham Forest Chelsea 31.12.2022
Chelsea OK
17 Tottenham Aston Villa 31.12.2022
Aston Villa OK
18 West Ham Brentford 31.12.2022
Brentford OK
19 Wolverhampton Wand

Manchester City OK
154 Crystal Palace Everton 22.04.2023
Everton OK
155 Fulham Leeds 22.04.2023
Leeds OK
156 Leicester Wolverhampton Wanderers 22.04.2023
Wolverhampton Wanderers OK
157 Liverpool Nottingham Forest 22.04.2023
Error Nottingham Forest
158 Manchester United Chelsea 22.04.2023
Chelsea OK
159 Newcastle United Tottenham 22.04.2023
Tottenham OK
160 Everton Newcastle United 25.04.2023
Newcastle United OK
161 Leeds Leicester 25.04.2023
Leicester OK
162 Nottingham Forest Brighton 25.04.2023
Brighton OK
163 Tottenham Manchester United 25.04.2023
Manchester United OK
164 West Ham Liverpool 25.04.2023
Liverpool OK
165 Wolverhampton Wanderers Crystal Palace 25.04.2023
Crystal Palace OK
166 Aston Villa Fulham 25.04.2023
Error Fulham
167 Chelsea Brentford 26.04.2023
Brentford OK
168 Southampton Bournemouth 26.04.2023
Error Bournemouth
169 Manchester City Arsenal 26.04.2023
Arsenal OK
170 Arsenal Chelsea 29.04.2023
Chelsea OK
171 Bournemouth Leeds 29.04.2023
Leeds OK
172 Brentford Nottin

In [31]:
temp_table[['Date', 'Host', 'Guest',
       'Host: G average at home last season',
       'Host: G average as guests last season', 'Host: GA average last season',
       'Host: G average last season', 'Host: league positon last season',
       'Host: GA 1-15 min last season', 'Host: GA 16-30 min last season',
       'Host: GA 31-45 min last season', 'Host: GA 46-60 min last season',
       'Host: GA 61-75 min last season', 'Host: GA 76+ min last season',
       'Guest: G average at home last season',
       'Guest: G average as guests last season',
       'Guest: G average last season', 'Guest: GA average last season',
       'Guest: league position last season', 'Guest: GA 1-15 min last season',
       'Guest: GA 16-30 min last season', 'Guest: GA 31-45 min last season',
       'Guest: GA 46-60 min last season', 'Guest: GA 61-75 min last season']].head()

,Date,Host,Guest,Host: G average at home last season,Host: G average as guests last season,Host: GA average last season,Host: G average last season,Host: league positon last season,Host: GA 1-15 min last season,Host: GA 16-30 min last season,...,Guest: G average at home last season,Guest: G average as guests last season,Guest: G average last season,Guest: GA average last season,Guest: league position last season,Guest: GA 1-15 min last season,Guest: GA 16-30 min last season,Guest: GA 31-45 min last season,Guest: GA 46-60 min last season,Guest: GA 61-75 min last season
0,26.12.2022,Arsenal,West Ham,1.842105,1.368421,1.263158,1.605263,5.0,0.196721,0.229508,...,1.736842,1.421053,1.578947,1.342105,7.0,0.083333,0.150000,0.183333,0.166667,0.216667
1,26.12.2022,Aston Villa,Liverpool,1.526316,1.210526,1.421053,1.368421,14.0,0.115385,0.115385,...,2.578947,2.368421,2.473684,0.684211,2.0,0.127660,0.180851,0.148936,0.127660,0.191489
2,26.12.2022,Brentford,Tottenham,1.157895,1.368421,1.473684,1.263158,13.0,0.104167,0.145833,...,2.000000,1.631579,1.815789,1.052632,4.0,0.159420,0.130435,0.173913,0.188406,0.202899
3,26.12.2022,Chelsea,Bournemouth,1.947368,2.052632,0.868421,2.000000,3.0,0.144737,0.144737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26.12.2022,Crystal Palace,Fulham,1.421053,1.210526,1.210526,1.315789,12.0,0.140000,0.080000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# статистика текущего сезона
# Host: G average this season, 'Host: GA average this season', 'Host: matches played this season', 
# 'Host: days since last match', 'Host: G/GA delta this season', 'Host: xG average this season',
# Guest: G average this season, 'Guest: GA average this season', 'Guest: matches played this season', 
# 'Gues: days since last match', 'Guest: G/GA delta this season', 'Guest: xG average this season' 

In [6]:
t_season = int(Ofstat.season_mapping[season.value].split('-')[0])
for index, row in temp_table.iterrows():
    
    print(index, row['Host'], row['Guest'], row['Date'])
    session = aiohttp.ClientSession()
    understat = Understat(session)

    host_stats = await understat.get_team_results(row['Host'], t_season)
    guest_stats = await understat.get_team_results(row['Guest'], t_season)
    
    host_upcoming = await understat.get_team_fixtures(row['Host'], t_season)
    guest_upcoming = await understat.get_team_fixtures(row['Guest'], t_season)
    #pd.DataFrame(host_stats)   
    
    host_stats=pd.DataFrame(host_stats)
    guest_stats=pd.DataFrame(guest_stats)
    host_upcoming=pd.DataFrame(host_upcoming)
    guest_upcoming=pd.DataFrame(guest_upcoming)
    
    host_prematch_stats =  my_understat.get_team_prematch_stats(row['Host'], pd.DataFrame(host_stats),host_upcoming, row['Date'], is_future=True)
    guest_prematch_stats =  my_understat.get_team_prematch_stats(row['Guest'], pd.DataFrame(guest_stats),guest_upcoming, row['Date'], is_future=True)

    for k in guest_prematch_stats.keys():
         temp_table.loc[index, k.replace('Team', 'Guest')] = guest_prematch_stats[k]

    for k in host_prematch_stats.keys():
         temp_table.loc[index, k.replace('Team', 'Host')] = host_prematch_stats[k]
        
    session.close()

0 Arsenal West Ham 26.12.2022


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000019568371340>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x00000195681A9AC0>, 109060.578)]']
connector: <aiohttp.connector.TCPConnector object at 0x00000195683713A0>


Last match played Arsenal against Wolverhampton Wanderers on 2022-11-12 19:45:00
Last match played Leicester against West Ham on 2022-11-12 15:00:00
1 Aston Villa Liverpool 26.12.2022


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001956850DB50>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x00000195683908E0>, 109062.265)]']
connector: <aiohttp.connector.TCPConnector object at 0x000001956850DD90>


Last match played Aston Villa against Brighton on 2022-11-13 14:00:00
Last match played Southampton against Liverpool on 2022-11-12 15:00:00
2 Brentford Tottenham 26.12.2022


...

Last match played Leeds against West Ham on 2023-05-20 11:00:00
Last match played Brentford against Tottenham on 2023-05-20 11:00:00
217 Leicester West Ham 28.05.2023


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000019568721820>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x000001956868D8E0>, 109428.937)]']
connector: <aiohttp.connector.TCPConnector object at 0x0000019568721C10>


Last match played Leicester against Newcastle United on 2023-05-20 11:00:00
Last match played Leeds against West Ham on 2023-05-20 11:00:00
218 Manchester United Fulham 28.05.2023


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000019568810F10>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x000001956868D8E0>, 109430.625)]']
connector: <aiohttp.connector.TCPConnector object at 0x0000019568721250>


Last match played Manchester United against Bournemouth on 2023-05-20 11:00:00
Last match played Crystal Palace against Fulham on 2023-05-20 11:00:00
219 Southampton Liverpool 28.05.2023
Last match played Southampton against Brighton on 2023-05-20 11:00:00
Last match played Aston Villa against Liverpool on 2023-05-20 11:00:00


In [10]:
temp_table[['Date', 'Host', 'Guest', 'Guest: G average this season', 'Guest: GA average this season',
       'Guest: matches played this season', 'Guest: days since last match',
       'Guest: G/GA delta this season', 'Guest: xG average this season',
       'Host: G average this season', 'Host: GA average this season',
       'Host: matches played this season', 'Host: days since last match',
       'Host: G/GA delta this season', 'Host: xG average this season']].head()

,Date,Host,Guest,Guest: G average this season,Guest: GA average this season,Guest: matches played this season,Guest: days since last match,Guest: G/GA delta this season,Guest: xG average this season,Host: G average this season,Host: GA average this season,Host: matches played this season,Host: days since last match,Host: G/GA delta this season,Host: xG average this season
0,26.12.2022,Arsenal,West Ham,0.800000,1.133333,15.0,44,-5.0,1.168691,2.357143,0.785714,14.0,44,22.0,0.843630
1,26.12.2022,Aston Villa,Liverpool,2.000000,1.214286,14.0,44,11.0,1.649719,1.066667,1.466667,15.0,43,-6.0,1.422414
2,26.12.2022,Brentford,Tottenham,2.066667,1.400000,15.0,44,10.0,1.204364,1.533333,1.666667,15.0,44,-2.0,1.508904
3,26.12.2022,Chelsea,Bournemouth,1.200000,2.133333,15.0,44,-14.0,1.693928,1.214286,1.214286,14.0,44,0.0,1.561856
4,26.12.2022,Crystal Palace,Fulham,1.600000,1.733333,15.0,43,-2.0,2.173699,1.071429,1.285714,14.0,44,-3.0,1.447139


In [29]:
for index, row in temp_table.iterrows():
    print(index, row['Host'], row['Guest'], row['Date'])
    current_match_date = row['Date']

    session = aiohttp.ClientSession()
    understat = Understat(session)

    end_date = reformat(current_match_date)

    league_table_on_date =  await understat.get_league_table(league_name=uliga, season=t_season, with_headers=True, end_date=end_date)
    session.close()

    league_table_on_date = my_understat.trim_async_table(league_table_on_date)
    
    try:
        team_stats_host = my_understat.get_team_stats_ondate(league_table_on_date, row['Host'])
        team_stats_host.pop('Team')
    except:
        print(f'No info on {row["Host"]}')
    try:
        team_stats_guest = my_understat.get_team_stats_ondate(league_table_on_date, row['Guest'])
        team_stats_guest.pop('Team')
    except:
         print(f'No info on {row["Guest"]}')
    for key in team_stats_host:
        temp_table.loc[index, f'Host: {key} this season'] = team_stats_host[key]
    
    for key in team_stats_guest:
        temp_table.loc[index, f'Guest: {key} this season'] = team_stats_guest[key]

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000195688841F0>


0 Arsenal West Ham 26.12.2022


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000195684132B0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x0000019569A33160>, 116835.343)]']
connector: <aiohttp.connector.TCPConnector object at 0x00000195684132E0>


1 Aston Villa Liverpool 26.12.2022


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000019568463850>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x0000019569A33B20>, 116836.234)]']
connector: <aiohttp.connector.TCPConnector object at 0x0000019568463040>


...

217 Leicester West Ham 28.05.2023


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001956AC81FA0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x0000019568649640>, 117082.937)]']
connector: <aiohttp.connector.TCPConnector object at 0x000001956AC816D0>


218 Manchester United Fulham 28.05.2023


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000195683CAA30>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x00000195686128E0>, 117084.046)]']
connector: <aiohttp.connector.TCPConnector object at 0x00000195683CA430>


219 Southampton Liverpool 28.05.2023


In [30]:
# пусто
temp_table[['Date', 'Host', 'Guest','Host: M this season', 'Host: W this season', 'Host: D this season',
       'Host: L this season', 'Host: G this season', 'Host: GA this season',
       'Host: PTS this season', 'Host: xG this season',
       'Host: NPxG this season', 'Host: xGA this season',
       'Host: NPxGA this season', 'Host: NPxGD this season',
       'Host: PPDA this season', 'Host: OPPDA this season',
       'Host: DC this season', 'Host: ODC this season',
       'Host: xPTS this season', 'Guest: M this season',
       'Guest: W this season', 'Guest: D this season', 'Guest: L this season',
       'Guest: G this season', 'Guest: GA this season',
       'Guest: PTS this season', 'Guest: xG this season',
       'Guest: NPxG this season', 'Guest: xGA this season',
       'Guest: NPxGA this season', 'Guest: NPxGD this season',
       'Guest: PPDA this season', 'Guest: OPPDA this season',
       'Guest: DC this season', 'Guest: ODC this season',
       'Guest: xPTS this season']].head()

,Date,Host,Guest,Host: M this season,Host: W this season,Host: D this season,Host: L this season,Host: G this season,Host: GA this season,Host: PTS this season,...,Guest: xG this season,Guest: NPxG this season,Guest: xGA this season,Guest: NPxGA this season,Guest: NPxGD this season,Guest: PPDA this season,Guest: OPPDA this season,Guest: DC this season,Guest: ODC this season,Guest: xPTS this season
0,26.12.2022,Arsenal,West Ham,14.0,12.0,1.0,1.0,33.0,11.0,37.0,...,19.59,16.55,17.53,15.25,1.30,18.37,15.50,66.0,83.0,21.75
1,26.12.2022,Aston Villa,Liverpool,15.0,5.0,3.0,7.0,16.0,22.0,18.0,...,29.58,29.58,23.10,20.81,8.77,10.30,21.39,165.0,97.0,22.44
2,26.12.2022,Brentford,Tottenham,15.0,4.0,7.0,4.0,23.0,25.0,19.0,...,26.09,23.80,18.07,17.30,6.50,13.52,16.04,107.0,114.0,25.89
3,26.12.2022,Chelsea,Bournemouth,14.0,6.0,3.0,5.0,17.0,17.0,21.0,...,13.32,13.32,25.41,21.60,-8.28,18.58,8.51,75.0,149.0,13.09
4,26.12.2022,Crystal Palace,Fulham,14.0,5.0,4.0,5.0,15.0,18.0,19.0,...,21.51,17.71,32.61,30.32,-12.62,15.23,11.90,71.0,116.0,15.32


In [ ]:
# h2h
# 'Host: h2h average xG 3y', 'Guest: h2h average xG 3y, 'Host: h2h average G 3y', 'Host: h2h average GA 3y'

In [32]:
for index, row in temp_table.iterrows():
    t_minus_1_season = int(Ofstat.season_mapping[season.value-1].split('-')[0])
    t_minus_2_season = int(Ofstat.season_mapping[season.value-2].split('-')[0])
    t_minus_3_season = int(Ofstat.season_mapping[season.value-3].split('-')[0])

    print(index, row['Host'], row['Guest'], row['Date'])
    session = aiohttp.ClientSession()
    understat = Understat(session)
    h2h_3_years = pd.DataFrame(columns=['target team','opposite team','date','target team goals','target team xG','opposite team goals','opposite team xG'])
    
    for year in [t_minus_1_season, t_minus_2_season, t_minus_3_season]:
        host_stats_year = await understat.get_team_results(row['Host'], year)
        host_stats_year = pd.DataFrame(host_stats_year)
        year_h2h = my_understat.get_season_h2h(host_stats_year, row["Guest"])
        h2h_3_years = h2h_3_years.append(year_h2h)
    
    temp_table.loc[index, 'Host: h2h average xG 3y'] = pd.DataFrame(h2h_3_years)['target team xG'].values.mean() if sum(pd.DataFrame(h2h_3_years)['target team xG'].values) != 0 else 0
    temp_table.loc[index, 'Guest: h2h average xG 3y'] = pd.DataFrame(h2h_3_years)['opposite team xG'].values.mean() if sum(pd.DataFrame(h2h_3_years)['opposite team xG'].values) != 0 else 0
    
    temp_table.loc[index, 'Host: h2h average G 3y'] = pd.DataFrame(h2h_3_years)['target team goals'].values.mean() if sum(pd.DataFrame(h2h_3_years)['target team goals'].values) != 0 else 0
    temp_table.loc[index, 'Host: h2h average GA 3y'] = pd.DataFrame(h2h_3_years)['opposite team goals'].values.mean() if sum(pd.DataFrame(h2h_3_years)['opposite team goals'].values) != 0 else 0

0 Arsenal West Ham 26.12.2022


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000019568463E20>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x0000019568486E20>, 117256.921)]']
connector: <aiohttp.connector.TCPConnector object at 0x00000195684637F0>


1 Aston Villa Liverpool 26.12.2022


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000019568827880>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x0000019569A5BDC0>, 117259.781)]']
connector: <aiohttp.connector.TCPConnector object at 0x0000019568827580>


2 Brentford Tottenham 26.12.2022


...

217 Leicester West Ham 28.05.2023


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000195684E74C0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x0000019569A1E880>, 117828.796)]']
connector: <aiohttp.connector.TCPConnector object at 0x00000195684E7550>


218 Manchester United Fulham 28.05.2023


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001956AB0BB20>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x000001956AB46BE0>, 117831.796)]']
connector: <aiohttp.connector.TCPConnector object at 0x000001956AB0B730>


219 Southampton Liverpool 28.05.2023


In [35]:
temp_table[['Date', 'Host', 'Guest', 'Host: h2h average xG 3y', 'Guest: h2h average xG 3y', 'Host: h2h average G 3y', 'Host: h2h average GA 3y']].head()

,Date,Host,Guest,Host: h2h average xG 3y,Guest: h2h average xG 3y,Host: h2h average G 3y,Host: h2h average GA 3y
0,26.12.2022,Arsenal,West Ham,1.635354,1.249815,2.166667,1.000000
1,26.12.2022,Aston Villa,Liverpool,1.142684,1.956815,1.666667,1.833333
2,26.12.2022,Brentford,Tottenham,0.770396,1.295117,0.000000,1.000000
3,26.12.2022,Chelsea,Bournemouth,2.001950,1.396360,1.000000,1.500000
4,26.12.2022,Crystal Palace,Fulham,1.449252,0.932857,1.000000,0.500000


In [38]:
# temp_table.to_csv('') имя файла куда сохранить

In [36]:
temp_table.to_csv('22_23_unlabeled_data_epl.csv')